# Lenet-5 Model with Deformable Convolution Layers

In [1]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.nn as nn
import random
import numpy as np
from torchvision.ops import deform_conv2d
from torchvision.ops import DeformConv2d
device = torch.device('cuda')
from torchvision.transforms import v2
from torch.utils.data import default_collate

### Import Fashion MNIST Dataset

In [2]:
# Download and load the MNIST dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform= ToTensor())
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=ToTensor())

### Set Seed

In [3]:
seed = 0

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Dataloaders with collate function for data augmentation using cutmix and mixup

In [4]:
batch_size = 50
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])
def collate_fn(batch):
    return cutmix_or_mixup(*default_collate(batch))

train_dataloader_augment = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn) # train data augment with Mixup and CutMix
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # normal training data
test_dataloader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)

### Define LeNet5 CNN with Deformable Convolution Layers

In [5]:

class LeNet5(nn.Module):
    def __init__(self, input_shape=(1, 28, 28), num_classes=10):
        super(LeNet5, self).__init__()
        torch.manual_seed(seed)
        self.offset1 = nn.Parameter(torch.randn(batch_size, 2*5*5, 24,24, requires_grad=True)*0.01)
        self.offset2 = nn.Parameter(torch.randn(batch_size, 2*5*5, 8,8, requires_grad=True)*0.01)
        
        self.deformconv1 = DeformConv2d(1, 32, kernel_size=5, stride=1) # Using DeformConv2d for conv layers
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.deformconv2 = DeformConv2d(32, 64, kernel_size=5, stride=1) # Using DeformConv2d for conv layers
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 * 4, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )

    def forward(self, x):
        x = self.deformconv1(x, self.offset1) # Using DeformConv2d for conv layers
        x = self.batchnorm1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.deformconv2(x, self.offset2) # Using DeformConv2d for conv layers
        x = self.batchnorm2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
loss_fn = nn.CrossEntropyLoss()
device = torch.device('cuda')

### Train Function

In [7]:
def train(train_loader, model, epochs, optimizer):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Calculate average training loss for the epoch
        avg_train_loss = train_loss / len(train_dataloader)
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}')

### Test Function

In [8]:
def test(test_loader, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = loss_fn(output, labels)
            test_loss += loss.item()

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy for the epoch
    avg_test_loss = test_loss / len(test_dataloader)
    accuracy = 100 * (correct / total)

    return avg_test_loss, accuracy

## Commence Training without data augmentation

In [9]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [10]:
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
train(train_dataloader, model, 50, optimizer)

Epoch 1/50, Train Loss: 0.4112
Epoch 2/50, Train Loss: 0.2835
Epoch 3/50, Train Loss: 0.2515
Epoch 4/50, Train Loss: 0.2216
Epoch 5/50, Train Loss: 0.2006
Epoch 6/50, Train Loss: 0.1790
Epoch 7/50, Train Loss: 0.1693
Epoch 8/50, Train Loss: 0.1545
Epoch 9/50, Train Loss: 0.1400
Epoch 10/50, Train Loss: 0.1303
Epoch 11/50, Train Loss: 0.1212
Epoch 12/50, Train Loss: 0.1139
Epoch 13/50, Train Loss: 0.1040
Epoch 14/50, Train Loss: 0.0971
Epoch 15/50, Train Loss: 0.0954
Epoch 16/50, Train Loss: 0.0905
Epoch 17/50, Train Loss: 0.0809
Epoch 18/50, Train Loss: 0.0793
Epoch 19/50, Train Loss: 0.0746
Epoch 20/50, Train Loss: 0.0720
Epoch 21/50, Train Loss: 0.0674
Epoch 22/50, Train Loss: 0.0669
Epoch 23/50, Train Loss: 0.0639
Epoch 24/50, Train Loss: 0.0608
Epoch 25/50, Train Loss: 0.0602
Epoch 26/50, Train Loss: 0.0611
Epoch 27/50, Train Loss: 0.0555
Epoch 28/50, Train Loss: 0.0530
Epoch 29/50, Train Loss: 0.0519
Epoch 30/50, Train Loss: 0.0501
Epoch 31/50, Train Loss: 0.0463
Epoch 32/50, Trai

### Test trained model without data augmentation on unseen data

In [12]:
avg_test_loss, acc = test(test_dataloader, model)

In [13]:
print("Accuracy on modified model: " + str(acc) + "%")
print("Average test loss on modified model: " + str(avg_test_loss))

Accuracy on modified model: 91.25%
Average test loss on modified model: 0.5138346299738623


## Commence Training with MixUp and CutMix augmentation

In [14]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [15]:
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
train(train_dataloader_augment, model, 50, optimizer)

Epoch 1/50, Train Loss: 1.1533
Epoch 2/50, Train Loss: 1.0097
Epoch 3/50, Train Loss: 0.9738
Epoch 4/50, Train Loss: 0.9472
Epoch 5/50, Train Loss: 0.9133
Epoch 6/50, Train Loss: 0.9089
Epoch 7/50, Train Loss: 0.9096
Epoch 8/50, Train Loss: 0.8882
Epoch 9/50, Train Loss: 0.8739
Epoch 10/50, Train Loss: 0.8917
Epoch 11/50, Train Loss: 0.8759
Epoch 12/50, Train Loss: 0.8522
Epoch 13/50, Train Loss: 0.8649
Epoch 14/50, Train Loss: 0.8572
Epoch 15/50, Train Loss: 0.8578
Epoch 16/50, Train Loss: 0.8454
Epoch 17/50, Train Loss: 0.8431
Epoch 18/50, Train Loss: 0.8396
Epoch 19/50, Train Loss: 0.8367
Epoch 20/50, Train Loss: 0.8360
Epoch 21/50, Train Loss: 0.8313
Epoch 22/50, Train Loss: 0.8328
Epoch 23/50, Train Loss: 0.8368
Epoch 24/50, Train Loss: 0.8268
Epoch 25/50, Train Loss: 0.8155
Epoch 26/50, Train Loss: 0.8281
Epoch 27/50, Train Loss: 0.8295
Epoch 28/50, Train Loss: 0.8216
Epoch 29/50, Train Loss: 0.8228
Epoch 30/50, Train Loss: 0.8262
Epoch 31/50, Train Loss: 0.8234
Epoch 32/50, Trai

### Test trained model with MixUp and CutMix augmentation on unseen data

In [17]:
avg_test_loss, acc = test(test_dataloader, model)

In [18]:
print("Accuracy on base model: " + str(acc) + "%")
print("Average test loss on base model: " + str(avg_test_loss))

Accuracy on base model: 92.23%
Average test loss on base model: 0.28454340070486067
